In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import BernoulliNB

In [2]:
new_df=[]
for cur,dir,files in os.walk('/kaggle/input/Alcoholics/SMNI_CMI_TRAIN'):
    for file in files:
        data=os.path.join(cur,file)
        
        new_df.append(data)

In [3]:
new_data=[]
for i in new_df:
    dg=pd.read_csv(i)
    new_data.append(dg)
    
    
new_data=pd.concat(new_data,axis=0)
new_data = new_data.sample(frac=1).reset_index(drop=True)


In [4]:
new_data.sample()


,Unnamed: 0,trial number,sensor position,sample num,sensor value,subject identifier,matching condition,channel,name,time
8244734,14263,12,PO8,123,7.833,a,S1 obj,55,co2a0000371,0.480469


In [5]:
new_data["subject identifier"].unique()

array(['c', 'a'], dtype=object)

In [6]:
new_data

,Unnamed: 0,trial number,sensor position,sample num,sensor value,subject identifier,matching condition,channel,name,time
0,5317,23,CP1,172,-1.902,c,S2 match,20,co2c0000338,0.671875
1,4560,45,C4,186,-6.561,c,"S2 nomatch,",17,co2c0000344,0.726562
2,4266,9,C3,149,-5.920,c,S2 match,16,co2c0000340,0.582031
3,1331,27,AF2,41,-0.203,a,S2 match,5,co2a0000364,0.160156
4,9905,43,FPZ,134,11.119,c,"S2 nomatch,",38,co2c0000339,0.523438
...,...,...,...,...,...,...,...,...,...,...
15335419,6018,3,P4,102,-9.135,a,S2 match,23,co2a0000368,0.398438
15335420,15731,3,CPZ,49,-2.543,c,S2 match,61,co2c0000338,0.191406
15335421,2969,5,FC2,137,-3.896,a,"S2 nomatch,",11,co2a0000365,0.535156
15335422,12534,12,CP3,193,-6.510,c,S1 obj,48,co2c0000338,0.753906


In [7]:
len(new_data)

15335424

In [8]:
label_encod=LabelEncoder()
Y=label_encod.fit_transform(new_data["subject identifier"])


In [9]:
transformer=ColumnTransformer(transformers=[
    ("oneHot",OneHotEncoder(sparse_output=False),["sensor position",'matching condition',])
])
TT=transformer.fit_transform(new_data)
TT=pd.DataFrame(TT,columns=transformer.get_feature_names_out()).reset_index(drop=True)


In [10]:
concaty = pd.concat([new_data,TT],axis=1)
concaty=concaty.drop(columns=["Unnamed: 0","sensor position","subject identifier","matching condition","name"])

In [11]:
# concaty.tail(10000)

In [12]:
Xt=concaty.iloc[0:5000000,:]
Yt=Y[0:5000000]
Xt2=concaty.iloc[5000000:10000000,:]
Yt2=Y[5000000:10000000]
Xt3=concaty.iloc[10000000:15000000,:]
Yt3=Y[10000000:15000000]

In [13]:
X_test=concaty.iloc[15325424:15335423]
Y_test=Y[15325424:15335423]

In [14]:
np.unique(Yt)

array([0, 1])

In [15]:
# random_classif=RandomForestClassifier()
# random_classif.fit(Xt,Yt)
# random_classif.score(X_test,Y_test)

In [16]:
sgd_classif=SGDClassifier()
sgd_classif.partial_fit(Xt,Yt,classes=np.unique(Yt))
sgd_classif.score(X_test,Y_test)

0.54005400540054

In [17]:
sgd_classif.partial_fit(Xt3,Yt3)
sgd_classif.score(X_test,Y_test)

0.504950495049505

In [18]:
sgd_classif.partial_fit(Xt2,Yt2)
sgd_classif.score(X_test,Y_test)

0.49334933493349337

In [19]:
bernoulli_classif=BernoulliNB()
bernoulli_classif.partial_fit(Xt,Yt,classes=np.unique(Yt))
bernoulli_classif.partial_fit(Xt2,Yt2)
bernoulli_classif.partial_fit(Xt3,Yt3)
bernoulli_classif.score(X_test,Y_test)

0.5073507350735074

In [20]:
y_pred=sgd_classif.predict(X_test)
np.unique(y_pred)

array([0, 1])

In [21]:
np.unique(Y_test)

array([0, 1])

In [22]:
Xt.shape

(5000000, 72)

In [15]:
from keras.layers import Dense,Input,LSTM,Dropout,SimpleRNN,GRU,Flatten,TimeDistributed
from keras.models import Model

from keras.callbacks import EarlyStopping

2024-08-17 05:41:31.134131: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 05:41:31.134312: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 05:41:31.326667: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [16]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [17]:
inputs=Input(shape=(1,72))
simple_rnn=SimpleRNN(32,return_sequences=True)(inputs)
lstm_rnn=LSTM(32,return_sequences=True)(simple_rnn)
gru_rnn=GRU(32,return_sequences=False)(lstm_rnn)
# drop_rnn=Dropout(0.5)(gru_rnn)
# dense_rnn=TimeDistributed(Dense(32,activation="relu"))(drop_rnn)
# gru_rnn=GRU(32,return_sequences=False)(dense_rnn)

outputs=Dense(1,activation="sigmoid")(gru_rnn)
model=Model(inputs,outputs)

In [18]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [19]:
Xt_reshaped=Xt.values.reshape(Xt.shape[0],1,Xt.shape[1])
Xt2_reshaped=Xt2.values.reshape(Xt2.shape[0],1,Xt.shape[1])
Xt3_reshaped=Xt3.values.reshape(Xt3.shape[0],1,Xt3.shape[1])

# Yt.shape

In [20]:
# for xt,yt in [(Xt_reshaped,Yt),(Xt2_reshaped,Yt2),(Xt3_reshaped,Yt3)]:
#     model.fit(xt,yt,batch_size=32,epochs=50,validation_split=0.2,callbacks=[early_stopping])

In [ ]:
model.fit(Xt_reshaped,Yt,batch_size=32,epochs=50,validation_split=0.2,callbacks=[early_stopping])